**모델, 토크나이저 불러오기**

In [10]:
from keras_crf import CRFModel
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.text import tokenizer_from_json
import json

class CustomCRFModel(CRFModel):
    def __init__(self, model, units, **kwargs):
        super(CustomCRFModel, self).__init__(model, units, **kwargs)
        self.model = model
        self.units = units

    def get_config(self):
        config = super().get_config()
        return config

    @classmethod
    def from_config(cls, config):
        return cls(**config)


# 저장된 모델 불러오기
model = load_model('bilstm_crf.h5', custom_objects={'CustomCRFModel': CustomCRFModel})

# 저장된 토크나이저 불러오기
with open('sentence_tokenizer_json.json', 'r', encoding='utf-8') as f:
    sentence_tokenizer_json = json.load(f)
    sentence_tokenizer = tokenizer_from_json(sentence_tokenizer_json)

with open('tag_tokenizer_json.json', 'r', encoding='utf-8') as f:
    tag_tokenizer_json = json.load(f)
    tag_tokenizer = tokenizer_from_json(tag_tokenizer_json)

In [13]:
index_to_word = sentence_tokenizer.index_word
index_to_tag = tag_tokenizer.index_word

In [14]:
text = """다시 태어난다면 다시 사랑한다면
그때는 우리 이러지 말아요
조금 덜 만나고 조금 덜 기대하며 많은 약속 않기로 해요
다시 이별이 와도 서로 큰 아픔 없이 돌아설 수 있을
만큼
버려도 되는 가벼운 추억만 서로의 가슴에 만들기로 해요
이젠 알아요 너무 깊은 사랑은 외려 슬픈 마지막을 가져온다는 걸 그대여 빌게요
다음 번에 사랑은 우리 같지 않길
부디 아픔이 없이
나나 꼭 나보다 더 행복해져야만해
많은 시간이 흘러 서로 잊고 지내도
지난날을 회상하며 그때도 이건 사랑이었다고 말할 수 있다면 그걸로 된 거죠
이젠 알아요 너무 깊은 사랑은 외려 슬픈 마지막을 가져온다는 걸
그대여 빌게요"""

text = text.split(" ")

In [15]:
X = sentence_tokenizer.texts_to_sequences([text])

In [17]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

max_len = 70

X = pad_sequences(X, padding='post', maxlen=max_len)

In [ ]:
y_predicted = model.predict(X)[0]

In [ ]:
print("{:20}|{:5}|{}".format("단어", "실제값", "예측값"))
print("-" * 38)

for pred in y_predicted[0]:
    if pred != 0: # PAD값은 제외
        print(index_to_tag[pred])